In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session~
#a lward ifires djilali d ahwawi ihemmey adihewes

/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/benchmark.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv
/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/train.csv', index_col='ID_ARBRE')
prev_data = pd.read_csv('/kaggle/input/are-sd-2025-classification-diagnostique-d-arbres/prev.csv', index_col='ID_ARBRE')
prev_data.head(3)

,quartier,site,genre_arbre,situation,type_sol,surf_permeable,classe_age,hauteur,classe_hauteur,diametre,...,fissure_houppier,ecorce_incluse_houppier,bois_mort_houppier,plaie_houppier,observation_houppier,esperance_maintien,contrainte,Long,Lat,Usage
ID_ARBRE,,,,,,,,,,,,,,,,,,,,,
559,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,44.563384,...,HPF,Non,HPBM,HPLNC,Branches cassées,2.0,Non,2.084,48.904,Private
560,Quartier 4 - Rotondes - St Léger,Avenue Saint-Fiacre,Carpinus,Alignement,Gr,1.0,JA,800,H2,38.197186,...,HPF,Non,HPBM,HPLS,0,1.0,Non,2.072,48.895,Private
561,Quartier 2 - Alsace - Pereire,Avenue du Maréchal Foch,Tilia,Alignement,MA,4.0,A,800,H2,63.661977,...,HPF,Non,HPBM,HPLS,0,2.0,Non,2.083,48.904,Private


***Code***

In [4]:
import numpy  as np
import pandas as pd 

def label_encoder(feature_vector):
    """
    Encode les catégories/étiquettes des données (en texte) en entier
    """
    unique_labels, encoded_labels = np.unique(feature_vector, return_inverse=True)
    return encoded_labels


def gini_impurity(labels : np.ndarray):
    """
    Calcul l'impureté de Gini pour un ensemble de prédictions.

    L'idée de l'impureté de Gini est la probabilité de se tromper dans une prédiction si on la prenanit au hasard au sein
    d'un échantillon donné.

    Entrées :
        labels : Un tableau de prédiction

    Sortie:
        La valeur d'impureté de Gini (float)
    """
    _, counts = np.unique(labels, return_counts=True)
    probabilities = counts/len(labels)
    return 1.-np.sum(probabilities**2)


def split_data( train_data : np.ndarray, labels : np.ndarray, feature_index : int, threshold : float):
    """
    Partitionne un ensemble d'entraînement (et de réponse) par rapport à une valeur de seuil.

    Entrée:
        train_data    : L'ensemble des données d'entraînement
        labels        : Les réponses correspondantes aux données d'entraînement
        feature_index : L'index du l'étiquette par rapport à laquelle on effectue le partitionnement
        threshold     : Le seuil définissant le partitionnement
    """
    left_mask  = train_data[:, feature_index] < threshold
    right_mask = ~left_mask
    return train_data[left_mask], labels[left_mask], train_data[right_mask], labels[right_mask]


def find_best_split(train_data : np.ndarray, labels : np.ndarray):
    """
    Trouve le meilleur partitionnement pour un ensemble de données d'entrée

    Entrées:
        train_data : Les données d'entrée
        labels     : Les prédictions correspondant à chaque entrée de l'entraînement
    
    Sortie:
        La meilleur étiquette (son index en fait) et la valeur de seuil pour son partitionnement
    """
    best_feature, best_threshold, best_gini = None, None, float('inf')
    for feature_index in range(train_data.shape[1]):
        # Récupère toutes les valeurs mesurées par l'étiquette courante (dans la boucle)
        values = np.unique(train_data[:, feature_index])
        # Recherche du seuil optimal pour cette étiquette :
        for threshold in values:
            _, left_labels, _, right_labels = split_data(train_data, labels, feature_index, threshold)
            if len(left_labels) == 0 or len(right_labels) == 0: continue
            # Calcul de l'indice de gini:
            gini = (len(left_labels)*gini_impurity(left_labels) + len(right_labels)*gini_impurity(right_labels)) / len(labels)
            # Plus l'indice de gini est petit, mieux c'est
            if gini < best_gini:
                best_feature, best_threshold, best_gini = feature_index, threshold, gini
    return best_feature, best_threshold


def build_tree(train_data : np.ndarray, labels : np.ndarray, max_depth : int, min_samples_split : int, depth : int =  0):
    """
    Construit un arbre de décision récursivement

    Entrées :
        train_data        : Les données d'entrée
        labels            : Les prédictions correspondant à chaque entrée de l'entraînement
        max_depth         : La profondeur maximale de l'arbre
        min_samples_split : Le nombre d'échantillons minimal requis pour partitionner un noeud de l'arbre
        depth (optionnel) : La profondeur de l'arbre actuellement traitée (dans la récursion)

    Sortie:
        L'arbre de décision retourné sous la forme d'un dictionnaire
    """
    if depth == max_depth or len(labels) < min_samples_split or gini_impurity(labels) == 0:
        return {'prediction' : np.argmax(np.bincount(labels))}
    feature, threshold = find_best_split(train_data, labels)
    # Si pas trouvé de feature adéquat, on arrête la construction de la branche et
    # on recherche la valeur la plus probable comme réponse (sous forme d'indice) :
    if feature is None: return {"prediction" : np.argmax(np.bincount(labels))}
    left_train_data, left_labels, right_train_data, right_labels = split_data(train_data, labels, feature, threshold)
    return {
        "feature"   : feature,
        "threshold" : threshold,
        "left"      : build_tree(left_train_data, left_labels, max_depth, min_samples_split, depth+1),
        "right"     : build_tree(right_train_data, right_labels, max_depth, min_samples_split, depth+1)
    }

def predict(tree : dict, data_point : np.ndarray):
    """
    Fait une prédiction pour un simple échantillon utilisant l'arbre de décision

    Entrées:
        tree       : L'arbre de décision représentée comme un dictionnaire
        data_point : L'échantillon d'entrée

    Sortie:
        La valeur prédite pour l'étiquette
    """
    if 'prediction' in tree:
        return tree['prediction']
    if data_point[tree['feature']] < tree['threshold']:
        return predict(tree['left'], data_point)
    else:
        return predict(tree['right'], data_point)
    

In [5]:
def predict_tree(data_prev: pd.DataFrame, data_train: pd.DataFrame, vvar: list, max_depth: int, min_samples_split: int):
    """ Predit la classification de tous les arbre de data_prev, avec comme données d'entrainement 
    les observations dans data_train en utilisant un seul arbre de décision avec comme paramètres : 
    mas_depth et min_samples_split en se basant sur les variables : vvars.
    """
    #construction de data_train_for_tree un nd.array qui peut être utilisé dans tree
    data_train_for_tree = np.zeros((len(data_train), len(vvar)))
    for (i, var) in zip(range(len(vvar)), vvar):
        #le cas: var est une variable qualitative
        if type(data_train.iloc[0][var]) == str:
            data_train_for_tree[:, i] = label_encoder( data_train[var] )
        else:
            data_train_for_tree[:, i] = data_train[var].to_numpy()

    #vecteur (nd.array) contenant les classes des arbres de data_train
    data_res = label_encoder(data_train["classification_diagnostic"])
    
    #creation de l'arbre
    tree = build_tree(data_train_for_tree, data_res, max_depth, min_samples_split)

    #Les predictions a faire:
    obs_a_predire = np.zeros((len(data_prev), len(vvar)), dtype= np.int64)
    for j, var in zip(range(len(vvar)), vvar):
        if type(data_prev.iloc[0][var]) == str:
            #print(label_encoder(data_prev[var]) )
            obs_a_predire[:, j] = label_encoder(data_prev[var]) 
        else:
            obs_a_predire[:, j] = data_prev[var].to_numpy()
    pred_res = np.array(['  ' for _ in range(len(data_prev))])
    for i, obs in zip(range(len(data_prev)), obs_a_predire):
        pred_res[i] = ("C" + str( predict( tree, obs ) +1))
    return pred_res


In [6]:
def random_forest (data_prev: pd.DataFrame, data_train: pd.DataFrame, vvar: list, max_depth: int, min_samples_split: int, nb_trees : int, nb_vars):
    """ Predit la classification de tous les arbre de data_prev, avec comme données d'entrainement 
    les observations dans data_train en utilisant une forêt aléatoire de nb_trees arbres de décision avec comme paramètres : 
    mas_depth et min_samples_split en se basant sur nb_vars variables de vvars choisies aléatoirement pour chaque arbre.
    """
    if len(vvar) < nb_vars:
        print("ERROR : len(vvar) < nb_vars (boucle infinie) len(varrs) = ", len(vvar))
        return 0
        
    train_long = (4*len(data_train))//5
    test_long = len(data_prev)
    
    votes = np.full((test_long, nb_trees), "", dtype='U2')
    ind = np.random.permutation(len(vvar))
    for i in range(nb_trees):
        vars_choisies_ind = np.random.permutation(len(vvar))
        vars_choisies = [vvar[i] for i in vars_choisies_ind[0:nb_vars]]
        #point a ameliorer : mettre le pourcentage de data_train a prendre en parametres
        v_rd = np.random.permutation(data_train.index)        
        sous_data_train = data_train[data_train.index.isin(v_rd[:train_long])]
        votes[:, i] = predict_tree(data_prev, sous_data_train, vars_choisies, max_depth, min_samples_split)
    v_res = np.full(test_long, "",dtype='U2')
    for i in range(test_long):
        val, counts = np.unique(votes[i], return_counts = True)
        v_res[i] = val[np.argmax(counts)]
    return v_res

***Entrainement***

In [7]:
def validation_croisée_tree(data: pd.DataFrame, predi_fonc, v_vars: np.array ,max_depth: int, min_samples_split: int, n_iter: int):
    """Renvoie le taux de réussite de la prediction du niveau de blessure des arbres par la fonction predi_f 
    testé sur data_test.
    """
    long = len(data) // 5
    res = []
    for i in range(n_iter):
        v_rd = np.random.permutation(data.index)
        data_test = data[data.index.isin(v_rd[:long])]
        data_train = data[data.index.isin(v_rd[long:])]
        prediction = predi_fonc(data_test, data_train, v_vars, max_depth, min_samples_split)
        realite = data_test["classification_diagnostic"].to_numpy()
        res.append(np.mean(prediction == realite) * 100)
    return np.mean(res)


In [8]:
def validation_croisée_foret(data: pd.DataFrame, predi_fonc, v_vars: np.array ,max_depth: int, min_samples_split: int, nb_trees: int, nb_vars: int, n_iter: int):
    """Renvoie le taux de réussite de la prediction du niveau de blessure des arbres par la fonction predi_f 
    testé sur data_test.
    """
    long = len(data) // 5
    res = []
    for i in range(n_iter):
        v_rd = np.random.permutation(data.index)
        data_test = data[data.index.isin(v_rd[:long])]
        data_train = data[data.index.isin(v_rd[long:])]
        prediction = predi_fonc(data_test, data_train, v_vars, max_depth, min_samples_split, nb_trees, nb_vars)
        realite = data_test["classification_diagnostic"].to_numpy()
        res.append(np.mean(prediction == realite) * 100)
    return np.mean(res)


In [9]:
vars_tout = ["circonference (en cm)", "surf_permeable","hauteur", "ecorce_incluse_houppier", "diametre", "port_arbre", "plaie_houppier", "fissure_tronc", "quartier", "contrainte"]

In [10]:
"""from tqdm import tqdm 

dico = {"accuracy" : [], "max_depth" : [], "min_samples_split" : [], "vars" : []}

for _ in tqdm(range(500)):
    rd = np.random.permutation(len(vars_tout))
    vars_ch = [vars_tout[i] for i in rd[:6]]
    for max_depth in [20, 25, 30, 35, 40, 45]:
        for min_samples_split in [5, 10, 15]:
            accuracy = validation_croisée_foret(data, random_forest, vars_ch,max_depth , min_samples_split, 50, 5, 30)
            dico["accuracy"].append(accuracy)
            dico["max_depth"].append(max_depth)
            dico["min_samples_split"].append(min_samples_split)
            dico["vars"].append(vars_ch)"""
        

'from tqdm import tqdm \n\ndico = {"accuracy" : [], "max_depth" : [], "min_samples_split" : [], "vars" : []}\n\nfor _ in tqdm(range(500)):\n    rd = np.random.permutation(len(vars_tout))\n    vars_ch = [vars_tout[i] for i in rd[:6]]\n    for max_depth in [20, 25, 30, 35, 40, 45]:\n        for min_samples_split in [5, 10, 15]:\n            accuracy = validation_croisée_foret(data, random_forest, vars_ch,max_depth , min_samples_split, 50, 5, 30)\n            dico["accuracy"].append(accuracy)\n            dico["max_depth"].append(max_depth)\n            dico["min_samples_split"].append(min_samples_split)\n            dico["vars"].append(vars_ch)'

In [11]:
vvars = ["circonference (en cm)", "surf_permeable","hauteur", "ecorce_incluse_houppier", "diametre", "contrainte"]
predi_tree = random_forest(prev_data, data, vvars, 15, 11, 50, 5)
#predi = random_forest(prev_data, data, vvars, 15, 11)
index_tree = np.array([i for i in prev_data.index])
v_res_tree = np.vstack((index_tree, predi_tree)).T
df_tree = pd.DataFrame(v_res_tree, columns=[['ID_ARBRE', 'classification_diagnostic']])
df_tree.to_csv("vpredi_tree.csv", index = False)

df_tree

,ID_ARBRE,classification_diagnostic
0,559,C2
1,560,C2
2,561,C2
3,562,C1
4,563,C4
...,...,...
132,694,C2
133,695,C1
134,696,C2
135,697,C1
